کتابخانه‌های مورد نیاز برای کار با تصاویر، شبکه‌های عصبی، پردازش و تقسیم داده

In [29]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

---


malimg dataset:

In [30]:
class MalimgDataset(Dataset):
    def __init__(self, root_dir):
        self.data = []
        self.labels = []
        all_labels = []

        for label in os.listdir(root_dir):
            folder = os.path.join(root_dir, label)
            if not os.path.isdir(folder):  # just folders
                continue

            for file in os.listdir(folder):
                img_path = os.path.join(folder, file)

                # just image files
                if not file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                    continue

                try:
                    img = Image.open(img_path).convert('L').resize((64, 64))  # grayscale + resize
                    self.data.append(np.array(img))
                    all_labels.append(label)
                except Exception as e:
                    print(f"error in openning the image file: {img_path}\n↳ {e}")

        # Encode labels
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        self.labels = le.fit_transform(all_labels)
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx].astype(np.float32) / 255.0  # Normalize to 0–1 and float
        y = self.labels[idx]
        x = torch.tensor(x).unsqueeze(0)  # Add channel dimension (1, 64, 64)
        y = torch.tensor(y).long()
        return x, y


convert('L') تبدیل عکس‌ها به سیاه وسفید توسط

استفاده میکنیمunsqueeze(0) برای اضافه کردن چنل از

برای نرمال سازی بین 0و1 بر 255 تقسیم میکنیم

---


CNN طراحی معماری

In [31]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)  # 64x64 → 64x64
        self.pool = nn.MaxPool2d(2, 2)                           # 64x64 → 32x32
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # 32x32 → 32x32 → 16x16

        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 25)  # hypothetically there is 25 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # conv1 + ReLU + maxpool
        x = self.pool(F.relu(self.conv2(x)))  # conv2 + ReLU + maxpool
        x = x.view(-1, 32 * 16 * 16)          # Flatten
        x = F.relu(self.fc1(x))              # FC1 + ReLU
        x = self.fc2(x)                      #  final result (Logits)
        return x


در این قسمت از کد تصاویر 64*64 را با یک کانال دریافت میکنیم و به 25 کلاس خروجی میدهیم

برای حفظ سایز استفاده میکنیم padding از دولایه کانولوشن با

استفاده میکنیمMaxPooling برای کاهش ابعاد از دولایه

با 128 نورون استفاده میکنیم Fully Connected از یک لایه

لایه خروجی 25 نورون (تعداد کلاس‌ها) دارد

است kernel_size=3 کوچک و رایج برای استخراج‌های محلی

مناسب برا یحفظ اندازه تصویر است padding=1

است filters=(16 and 32) overfitting تعداد مناسب برای شبکه ساده و بدون

---


استفاده از دیتاست آماده از منبع زیر

https://github.com/spoinguscar/malimg-dataset

---


DataLoader تقسیم داده و ساخت

In [32]:
# hyper parameters
batch_size = 32
learning_rate = 0.001

# loading datasets
path = r'E:\uni\cyberSecurity\HW7\A\malimg-dataset'
print(os.path.exists(path))
dataset = MalimgDataset(path)


# train and test
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# data loaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)


True


batch_size=32 رایج ترین مقدار برای تعادل بین سرعت و دقت

مناسب است learning_rate=0.001 برای شروع

---


optimizer and loss function تعریف مدل و

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

---


آموزش مدل

In [34]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}")


Epoch 1/10, Loss: 242.0686
Epoch 2/10, Loss: 42.6859
Epoch 3/10, Loss: 31.0928
Epoch 4/10, Loss: 25.0891
Epoch 5/10, Loss: 20.0346
Epoch 6/10, Loss: 18.5526
Epoch 7/10, Loss: 17.4679
Epoch 8/10, Loss: 15.9720
Epoch 9/10, Loss: 15.1091
Epoch 10/10, Loss: 13.5058


کافی و سریع است num_epochs=10 برای مشاهده روند آموزش

---


ارزیابی مدل روی داده تست

In [35]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 97.81%
